In [1]:
# HF Embedding
# Jason Miles

# A program to calculate the electronic structure of H2O, Pyridine, and Benzene
#   using the HF Embedding Scheme described in: https://arxiv.org/abs/2009.01872

In [2]:
from qiskit_nature.drivers import PySCFDriver, UnitsType
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem

from qiskit.algorithms.optimizers import L_BFGS_B, SLSQP
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.operators.second_quantization.qubit_converter import QubitConverter

from qiskit_nature.circuit.library import HartreeFock, UCCSD

from qiskit import Aer, IBMQ

from qiskit.algorithms import VQE

from qiskit_nature.transformers import ActiveSpaceTransformer

from qiskit.aqua.operators import Z2Symmetries

import matplotlib.pyplot as plt

/home/jay/.local/lib/python3.6/site-packages/qiskit/__init__.py:67: DeprecationWarning: Using Qiskit with Python 3.6 is deprecated as of the 0.17.0 release. Support for running Qiskit with Python 3.6 will be removed in a future release.
  "future release.", DeprecationWarning)


In [3]:
from qiskit_nature.operators.second_quantization.qubit_converter import QubitConverter

In [4]:
# this function is set up to calculate the energy of a molecule with 
# different active space configurations:
#   (n el, n orb)
def calc_hf_embed(molecule, backend, as_orb):

    # range for num electrons
    for el in range(2, 5):

        # set up driver
        # the driver encodes the molecule information
        driver = PySCFDriver(atom=molecule, unit=UnitsType.ANGSTROM, basis='631g')

        # active space:
        nel = 2*el
        norb = as_orb
        ast = ActiveSpaceTransformer(num_electrons=nel, num_molecular_orbitals=norb)
        print("\nActive Space:\nNum Electrons: {}\nNum Orbitals: {}".format(nel, norb))

        # problem
        problem = ElectronicStructureProblem(driver=driver, q_molecule_transformers=[ast])

        # calculate second quantized operators
        # this function will transform the molecule with the given active space
        sqops = problem.second_q_ops()
        main_op = sqops[0]

        # extract initial HF energy
        print("HF energy: ", problem.molecule_data_transformed.hf_energy, "\n")  

        # plot orbital energies
        plot_bool = False
        if plot_bool:
            plot_orbital_energies(problem.molecule_data.orbital_energies)
            plot_orbital_energies(problem.molecule_data_transformed.orbital_energies)

        # num_particles, or, num electrons
        num_parts = (problem.molecule_data_transformed.num_alpha, 
                            problem.molecule_data_transformed.num_beta)

        # spin orbitals hold one electron each, and will be mapped to qubits
        num_spin_orbs = 2 * problem.molecule_data_transformed.num_molecular_orbitals

        # other parameters of VQE
        optimizer = SLSQP()
        # optimizer = L_BFGS_B()
        mapper = ParityMapper()
        converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)
        
        qubit_op = converter.convert(main_op, num_particles=num_parts)

        # initial state of system
        # remember, this represents the Hamiltonian
        init_state = HartreeFock(num_spin_orbs, num_parts, converter)

        # ansatz
        # parameterized circuit representing wave function
        ansatz = UCCSD(qubit_converter=converter, num_particles=num_parts,
                        num_spin_orbitals=num_spin_orbs, initial_state=init_state)
        
        # algorithm
        algorithm = VQE(ansatz, optimizer, quantum_instance=backend)

        # results
        result = algorithm.compute_minimum_eigenvalue(qubit_op)
        electro_res = problem.interpret(result)
        print(electro_res)




In [ ]:

def plot_orbital_energies(energies):

    plt.scatter( [1] * len(energies), energies, marker='_' )
    plt.ylabel("Energy (HA)")
    plt.title("Orbital Energies")
    plt.show()




In [5]:
def calc_benzene(molecule, backend, as_orb):

    for el in range(3, 5):

        # set up driver
        # the driver encodes the molecule information
        driver = PySCFDriver(atom=molecule, unit=UnitsType.ANGSTROM, basis='631g')

        # active space:
        nel = 2*el
        # norb = 6
        norb = as_orb
        ast = ActiveSpaceTransformer(num_electrons=nel, num_molecular_orbitals=norb)
        print("\nActive Space:\nNum Electrons: {}\nNum Orbitals: {}".format(nel, norb))

        # problem
        problem = ElectronicStructureProblem(driver=driver, q_molecule_transformers=[ast])

        # calculate second quantized operators
        # this function will transform the molecule with the given active space
        sqops = problem.second_q_ops()
        main_op = sqops[0]

        # extract initial HF energy
        print("HF energy: ", problem.molecule_data_transformed.hf_energy, "\n")      

        # num_particles, or, num electrons
        num_parts = (problem.molecule_data_transformed.num_alpha, 
                            problem.molecule_data_transformed.num_beta)

        # spin orbitals hold one electron each, and will be mapped to qubits
        num_spin_orbs = 2 * problem.molecule_data_transformed.num_molecular_orbitals

        # other parameters of VQE
        optimizer = SLSQP()
        # optimizer = L_BFGS_B()
        mapper = ParityMapper()
        converter = QubitConverter(mapper=mapper, two_qubit_reduction=True, z2symmetry_reduction='auto')
        
        qubit_op = converter.convert(main_op, num_particles=num_parts)

        print("n qubs: ", qubit_op.num_qubits)

        # initial state of system
        # remember, this represents the Hamiltonian
        init_state = HartreeFock(num_spin_orbs, num_parts, converter)

        # ansatz
        # parameterized circuit representing wave function
        ansatz = UCCSD(qubit_converter=converter, num_particles=num_parts,
                        num_spin_orbitals=num_spin_orbs, initial_state=init_state)
        
        # algorithm
        algorithm = VQE(ansatz, optimizer, quantum_instance=backend)

        # results
        result = algorithm.compute_minimum_eigenvalue(qubit_op)

        electro_res = problem.interpret(result)
        print(electro_res)




In [7]:
# define backend

# IBMQ.load_account()
# provider = IBMQ.get_provider(hub='ibm-q')
# backend = provider.get_backend('simulator_statevector')

backend = Aer.get_backend('statevector_simulator')


In [8]:

h2o = 'O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0'

print("\nCalculating the energy of H2O...\n")
#calc_hf_embed(h2o, backend, 'h2o')



Calculating the energy of H2O...



In [9]:

pyridine = '''C 1.3603 0.0256 0.0000; C 0.6971 -1.2020 0.0000; 
            C -0.6944 -1.2184 0.0000; C -1.3895 -0.0129 0.0000; 
            C -0.6712 1.1834 0.0000; N 0.6816 1.1960 0.0000;
            H 2.4530 0.1083 0.0000; H 1.2665 -2.1365 0.0000; 
            H -1.2365 -2.1696 0.0000; H -2.4837 0.0011 0.0000; 
            H -1.1569 2.1657 0.0000'''

print("\nCalculating the energy of pyridine...\n")
#calc_hf_embed(pyridine, backend, 'pyridine')



Calculating the energy of pyridine...



In [10]:

benzene = '''C .0 1.403 .0; H .0 2.49 .0; C -1.215 0.701 .0; H -2.157 1.245 .0; 
        C -1.215 -.701 .0; H -2.157 -1.245 .0; C .0 -1.403 .0; H .0 -2.49 .0; 
        C 1.215 -.701 .0; H 2.157 -1.245 .0; C 1.215 .701 .0; H 2.157 1.245 .0'''

print("\nCalculating the energy of benzene...\n")
calc_benzene(benzene, backend, 7)



Calculating the energy of benzene...


Active Space:
Num Electrons: 6
Num Orbitals: 8
HF energy:  -230.62229793974825 

ipykernel_launcher:38: DeprecationWarning: This QubitConverter is deprecated as of 0.1.0, and will be removed no earlier than 3 months after the release. You should use the qiskit_nature.converters.second_quantization QubitConverter as a direct replacement instead.
n qubs:  14
spin orbs:  16
particles:  (3, 3)


AttributeError: 'NoneType' object has no attribute 'num_qubits'